# SyftBox SDK Development Testing

This notebook is for testing and validating the SyftBox SDK during development.

In [3]:
from syft_nsai_sdk.logger import get_logger

logger = get_logger()
logger.info("This is an informational message.")
logger.error("This is an error message.")

2025-08-14 18:27:47 - syft_nsai_sdk - INFO - This is an informational message.
2025-08-14 18:27:47 - syft_nsai_sdk - ERROR - This is an error message.


In [5]:
# Install the SDK in development mode (run this once)
# !pip install -e .

# Import the SDK
import syft_nsai_sdk as sb
logger.success("SyftBox SDK imported successfully")

2025-08-14 18:28:05 - syft_nsai_sdk - SUCCESS - SyftBox SDK imported successfully


## 1. Check SDK Status

In [7]:
# Check if SDK is properly configured
print("📊 Checking SDK status...")
status_info = sb.status()

logger.success(f"🔐 Authenticated: {sb.is_authenticated}")
if sb.current_user:
    logger.info(f"👤 Current user: {sb.current_user}")

2025-08-14 18:28:40 - syft_nsai_sdk.sdk - INFO - SyftBox SDK Status:
2025-08-14 18:28:40 - syft_nsai_sdk.sdk - INFO - SyftBox SDK Status:
2025-08-14 18:28:40 - syft_nsai_sdk.sdk - INFO -    Config Exists: Yes
2025-08-14 18:28:40 - syft_nsai_sdk.sdk - INFO -    Config Exists: Yes
2025-08-14 18:28:40 - syft_nsai_sdk.sdk - INFO -    Authenticated: Yes
2025-08-14 18:28:40 - syft_nsai_sdk.sdk - INFO -    Authenticated: Yes
2025-08-14 18:28:40 - syft_nsai_sdk.sdk - INFO -    User: callis@openmined.org
2025-08-14 18:28:40 - syft_nsai_sdk.sdk - INFO -    User: callis@openmined.org
2025-08-14 18:28:40 - syft_nsai_sdk.sdk - INFO -    Server: https://syftbox.net
2025-08-14 18:28:40 - syft_nsai_sdk.sdk - INFO -    Server: https://syftbox.net
2025-08-14 18:28:40 - syft_nsai_sdk.sdk - INFO -    Data Dir: /Users/callisezenwaka/SyftBox
2025-08-14 18:28:40 - syft_nsai_sdk.sdk - INFO -    Data Dir: /Users/callisezenwaka/SyftBox
2025-08-14 18:28:40 - syft_nsai_sdk - SUCCESS - 🔐 Authenticated: <function i

📊 Checking SDK status...


## 2. Discover Models

In [9]:
# Get all available models
logger.info("🔍 Discovering all models...")
all_models = sb.get_models()
logger.success(f"Found {len(all_models)} total models")

if all_models:
    logger.info("\nModel List:")
    sb.display_models(all_models)
else:
    logger.warning("No models found. Make sure SyftBox is running and synced.")

2025-08-14 18:29:49 - syft_nsai_sdk - INFO - 🔍 Discovering all models...
2025-08-14 18:29:49 - syft_nsai_sdk - SUCCESS - Found 10 total models
2025-08-14 18:29:49 - syft_nsai_sdk - INFO - 
Model List:


╭────────────────────┬──────────────────────────┬─────────────────────────────────┬──────────────┬───────────────────────────────────────────────────────┬───────────╮
│ Name               │ Owner                    │ Tags                            │ Services     │ Summary                                               │ Status    │
├────────────────────┼──────────────────────────┼─────────────────────────────────┼──────────────┼───────────────────────────────────────────────────────┼───────────┤
│ shubham-router     │ shubham@openmined.org    │ AI                              │ search       │ This is a short summary                               │ available │
├────────────────────┼──────────────────────────┼─────────────────────────────────┼──────────────┼───────────────────────────────────────────────────────┼───────────┤
│ the-city           │ speters@thecity.nyc      │ news                            │ search       │ The City                                              │ available 

## 3. Filter Models by Tags

In [5]:
# Search for models with specific tags
test_tags = ["news", "AI", "test", "chat", "ollama", "nlp", "search", "chat"]

for tag in test_tags:
    print(f"\nLooking for '{tag}' models...")
    tagged_models = sb.find_models(tags=[tag])
    print(f"Found {len(tagged_models)} models with '{tag}' tag")
    
    if tagged_models:
        for model in tagged_models:
            print(f"  - {model.name} by {model.owner} | Services: {model.services}")


Looking for 'news' models...
Found 2 models with 'news' tag
  - the-city by speters@thecity.nyc | Services: ['search']
  - ionesio-router by ionesio@openmined.org | Services: ['search']

Looking for 'AI' models...
Found 1 models with 'AI' tag
  - shubham-router by shubham@openmined.org | Services: ['search']

Looking for 'test' models...
Found 1 models with 'test' tag
  - callis by callis@openmined.org | Services: ['chat']

Looking for 'chat' models...
Found 1 models with 'chat' tag
  - claude-sonnet-3.5 by aggregator@openmined.org | Services: ['chat', 'search']

Looking for 'ollama' models...
Found 2 models with 'ollama' tag
  - public-tinnyllama by irina@openmined.org | Services: ['chat']
  - ollama-model by irina@openmined.org | Services: ['chat']

Looking for 'nlp' models...
Found 0 models with 'nlp' tag

Looking for 'search' models...
Found 0 models with 'search' tag

Looking for 'chat' models...
Found 1 models with 'chat' tag
  - claude-sonnet-3.5 by aggregator@openmined.org | S

## 4. Test Model Interaction

In [10]:
# Test model interaction if models are available
if all_models:
    test_model = all_models[0]
    print(f"🤖 Testing interaction with: {test_model.name}")
    print(f"   Owner: {test_model.owner}")
    print(f"   Services: {test_model.services}")
    print(f"   Tags: {test_model.tags}")
    print(f"   Status: {test_model.status}")
    
    # Test chat if available
    if "chat" in test_model.services:
        print("\n💬 Testing chat service...")
        try:
            response = test_model.chat(
                "Hello! This is a test message from the development notebook.",
                temperature=0.7,
                max_tokens=50
            )
            print(f"✅ Chat Response: {response}")
        except Exception as e:
            logger.error(f"❌ Chat failed: {e}")
    
    # Test search if available
    if "search" in test_model.services:
        print("\n🔍 Testing search service...")
        try:
            results = test_model.search("test query")
            logger.success(f"✅ Search Results: {results[:200]}...")
        except Exception as e:
            logger.error(f"❌ Search failed: {e}")
else:
    print("⚠️ No models available for testing")

🤖 Testing interaction with: shubham-router
   Owner: shubham@openmined.org
   Services: ['search']
   Tags: ['AI']
   Status: available

🔍 Testing search service...


2025-08-14 18:30:37 - syft_nsai_sdk - ERROR - ❌ Search failed: Failed to call service 'search': RPC request failed with status 401: {"code":"E_AUTH_INVALID_CREDENTIALS","error":"invalid access token: token signature is invalid: signature is invalid"}



## 5. Test Different Owner Filters

In [7]:
# Test filtering by specific owners
if all_models:
    unique_owners = list(set(model.owner for model in all_models))
    print(f"📊 Found {len(unique_owners)} unique model owners:")
    
    for owner in unique_owners[:3]:  # Test first 3 owners
        print(f"\n👤 Models by {owner}:")
        owner_models = sb.find_models(owners=[owner])
        for model in owner_models:
            print(f"  - {model.name} | {model.services}")

📊 Found 7 unique model owners:

👤 Models by callis@openmined.org:
  - callis | ['chat']

👤 Models by irina@openmined.org:
  - public-tinnyllama | ['chat']
  - openmined-about | ['search']
  - ollama-model | ['chat']

👤 Models by shubham@openmined.org:
  - shubham-router | ['search']


## 6. Performance Testing

In [8]:
import time

# Test discovery performance
print("⏱️ Performance Testing...")

start_time = time.time()
models = sb.get_models()
discovery_time = time.time() - start_time

print(f"📈 Model Discovery: {discovery_time:.2f}s for {len(models)} models")

if models:
    # Test individual model lookup performance
    test_model = models[0]
    
    start_time = time.time()
    found_model = sb.get_model(test_model.name, owner=test_model.owner)
    lookup_time = time.time() - start_time
    
    print(f"🔍 Model Lookup: {lookup_time:.2f}s")
    print(f"✅ Found: {found_model.name if found_model else 'Not found'}")

⏱️ Performance Testing...
📈 Model Discovery: 0.01s for 10 models
🔍 Model Lookup: 0.00s
✅ Found: shubham-router


## 7. Error Handling Tests

In [11]:
# Test error handling
print("Testing error handling...")

# Test non-existent model
try:
    fake_model = sb.get_model("non-existent-model-12345")
    print(f"Non-existent model result: {fake_model}")
except Exception as e:
    logger.error(f"❌ Expected error for non-existent model: {e}")

# Test invalid filters
try:
    empty_results = sb.find_models(tags=["non-existent-tag-12345"])
    print(f"✅ Invalid tag search returned {len(empty_results)} results (expected 0)")
except Exception as e:
    logger.error(f"❌ Unexpected error with invalid tags: {e}")

print("✅ Error handling tests completed")

Testing error handling...
Non-existent model result: None
✅ Invalid tag search returned 0 results (expected 0)
✅ Error handling tests completed


## 8. Development Notes

Use this section to test new features or debug issues:

In [ ]:
# Scratch space for development testing
print("🔧 Development testing area")

# Add your test code here
# Example:
# models = sb.find_models(name="specific-model")
# print(f"Specific search: {len(models)} results")